# Markdown to Chunked Dataset

This notebook converts the markdown dataset to a chunked dataset using header-based splitting. This approach preserves the document structure by keeping headings with their content, making chunks more meaningful and contextually rich.

In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

In [2]:
# ---------------- CONFIG ----------------
INPUT_FILE = "data/tawiki_markdown.parquet"
OUTPUT_FILE = "data/tawiki_chunked.parquet"

# Chunking parameters
CHUNK_SIZE = 8000          # Target chunk size in characters
CHUNK_OVERLAP = 200        # Overlap between chunks

# Headers to split on (for Tamil markdown)
HEADERS_TO_SPLIT = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]
# ---------------------------------------

In [3]:
# Load the markdown dataset
print(f"Loading data from {INPUT_FILE}...")
df = pd.read_parquet(INPUT_FILE)
print(f"Loaded {len(df)} documents")
df.head()

Loading data from data/tawiki_markdown.parquet...
Loaded 171015 documents


,text
0,# விக்கிப்பீடியா:கலந்துரையாடல்\n\n***இங்கு தமி...
1,# விக்கிப்பீடியா:புதுப் பயனர் பக்கம்\n\n\n\nவி...
2,# விக்கிப்பீடியா:விக்கிப்பீடியர்கள்\n\n\nவிக்க...
3,# கட்டடக்கலை\n\nசிறந்த மற்றும் மிகவும் நுட்பமா...
4,# கட்டிடங்களின் பட்டியல்\n\n\nபிரபலமான அல்லது ...


## Test with a single document

Let's test the chunking approach with one document to see how it works.

In [4]:
# Test with first document
sample_text = df['text'].iloc[1]
print(f"Sample document length: {len(sample_text)} characters")
print(f"\nFirst 500 characters:\n{sample_text[:500]}...")

# Initialize splitters
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=HEADERS_TO_SPLIT,
    strip_headers=True  # Strip headers, we'll add them back manually
)

# Split by headers first
header_splits = markdown_splitter.split_text(sample_text)

print(f"\n\nAfter header-based splitting: {len(header_splits)} sections")
for i, doc in enumerate(header_splits[:3]):
    print(f"\n--- Section {i+1} ---")
    # print(f"Metadata: {doc.metadata}")
    # print(f"Content preview: {doc.page_content[:200]}...")
    
    # Show how headers will be prepended
    header_prefix = ""
    for j in range(1, 5):
        header_key = f"Header {j}"
        if header_key in doc.metadata:
            header_prefix += f"{'#' * j} {doc.metadata[header_key]}\n\n"
    # update doc.page_content to include headers
    doc.page_content = f"{header_prefix}{doc.page_content}"
    
    print(f"\nWith headers prepended:\n{doc.page_content[:200]}...")
    print("--------------------------------------------------")


Sample document length: 3659 characters

First 500 characters:
# விக்கிப்பீடியா:புதுப் பயனர் பக்கம்



விக்கிப்பீடியா, அதன் வாசகர்களால் ஒருமித்து எழுதப்படுகின்ற ஓர் இலவசக் கலைக்களஞ்சியம் ஆகும். **நீங்கள்** உட்பட எவரும், ஒவ்வொரு விக்கிப்பீடியா கட்டுரைப் பக்கத்திலும் காணப்படும் **தொகு** இணைப்பைச் சொடுக்குவதன் மூலம், எந்தக் கட்டுரையையும் தொகுக்க முடியும்.  

## விக்கிப்பீடியாவில் உலாவுதல்
விக்கிப்பீடியாவின் தமிழ்ப் பதிப்பு 2003இல் தான் தொடங்கப்பட்டுள்ளது எனினும், ஆங்கிலத்திலும் ஏனைய பல மொழிகளிலும், பல்வேறு துறைகளையும் சார்ந்த ஏராளமான தகவல்களை, விக்கிப்பீடியா ஏ...


After header-based splitting: 3 sections

--- Section 1 ---

With headers prepended:
# விக்கிப்பீடியா:புதுப் பயனர் பக்கம்

விக்கிப்பீடியா, அதன் வாசகர்களால் ஒருமித்து எழுதப்படுகின்ற ஓர் இலவசக் கலைக்களஞ்சியம் ஆகும். **நீங்கள்** உட்பட எவரும், ஒவ்வொரு விக்கிப்பீடியா கட்டுரைப் பக்கத்திலும்...
--------------------------------------------------

--- Section 2 ---

With headers prepended:
# விக்கிப்பீடியா:புதுப் பயனர் பக்கம்

## விக்கி

In [5]:
# Apply character-level splitting for size constraint
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

# Split the header-based sections into smaller chunks
final_chunks = text_splitter.split_documents(header_splits)

print(f"After character-level splitting: {len(final_chunks)} chunks")
print(f"\nSample chunks:")
for i, chunk in enumerate(final_chunks[:3]):
    print(f"\n--- Chunk {i+1} ---")
    print(f"Metadata: {chunk.metadata}")
    print(f"Length: {len(chunk.page_content)} characters")
    print(f"Content: {chunk.page_content[:200]}...")


After character-level splitting: 3 chunks

Sample chunks:

--- Chunk 1 ---
Metadata: {'Header 1': 'விக்கிப்பீடியா:புதுப் பயனர் பக்கம்'}
Length: 289 characters
Content: # விக்கிப்பீடியா:புதுப் பயனர் பக்கம்

விக்கிப்பீடியா, அதன் வாசகர்களால் ஒருமித்து எழுதப்படுகின்ற ஓர் இலவசக் கலைக்களஞ்சியம் ஆகும். **நீங்கள்** உட்பட எவரும், ஒவ்வொரு விக்கிப்பீடியா கட்டுரைப் பக்கத்திலும்...

--- Chunk 2 ---
Metadata: {'Header 1': 'விக்கிப்பீடியா:புதுப் பயனர் பக்கம்', 'Header 2': 'விக்கிப்பீடியாவில் உலாவுதல்'}
Length: 997 characters
Content: # விக்கிப்பீடியா:புதுப் பயனர் பக்கம்

## விக்கிப்பீடியாவில் உலாவுதல்

விக்கிப்பீடியாவின் தமிழ்ப் பதிப்பு 2003இல் தான் தொடங்கப்பட்டுள்ளது எனினும், ஆங்கிலத்திலும் ஏனைய பல மொழிகளிலும், பல்வேறு துறைகளையும...

--- Chunk 3 ---
Metadata: {'Header 1': 'விக்கிப்பீடியா:புதுப் பயனர் பக்கம்', 'Header 2': 'தொகுத்தல்'}
Length: 2432 characters
Content: # விக்கிப்பீடியா:புதுப் பயனர் பக்கம்

## தொகுத்தல்

ஒவ்வொருவரும் விக்கிப்பீடியாவிலுள்ள பக்கங்களைத் தொகுக்கமுடியும் (இந்தப் பக்கத்தையும்

## Process all documents

Now let's process all documents and create the chunked dataset with metadata.

In [6]:
def process_document(text):
    """
    Process a single document: split by headers, then by character limit.
    Returns list of dicts with text and metadata.
    Headers are prepended BEFORE character-level splitting for better context.
    """
    # Initialize splitters
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=HEADERS_TO_SPLIT,
        strip_headers=True  # Strip from content, we'll add them back manually
    )
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    
    # Split by headers first
    try:
        header_splits = markdown_splitter.split_text(text)
    except Exception as e:
        # If header splitting fails, treat entire text as one document
        from langchain_core.documents import Document
        header_splits = [Document(page_content=text, metadata={})]
    
    # Prepend headers to each section BEFORE character splitting
    from langchain_core.documents import Document
    sections_with_headers = []
    
    for doc in header_splits:
        # Build header prefix from metadata
        header_prefix = ""
        for i in range(1, 5):  # Header 1 to Header 4
            header_key = f"Header {i}"
            if header_key in doc.metadata:
                header_prefix += f"{'#' * i} {doc.metadata[header_key]}\n\n"
        
        # Prepend headers to content
        text_with_headers = header_prefix + doc.page_content
        
        # Create new document with headers in content
        sections_with_headers.append(
            Document(page_content=text_with_headers, metadata=doc.metadata)
        )
    
    # Now split by character limit (headers are already in the content)
    final_chunks = text_splitter.split_documents(sections_with_headers)
    
    # Convert to dict format
    result = []
    for chunk in final_chunks:
        result.append({
            'text': chunk.page_content,
            'metadata': chunk.metadata
        })
    
    return result

In [7]:
# Process all documents
all_chunks = []

print(f"Processing {len(df)} documents...")
for text in tqdm(df['text']):
    chunks = process_document(text)
    all_chunks.extend(chunks)

print(f"\nTotal chunks created: {len(all_chunks)}")

# Create DataFrame
chunks_df = pd.DataFrame(all_chunks)
print(f"DataFrame shape: {chunks_df.shape}")
chunks_df.head()

Processing 171015 documents...


  0%|          | 0/171015 [00:00<?, ?it/s]

100%|██████████| 171015/171015 [02:21<00:00, 1206.26it/s]



Total chunks created: 532233
DataFrame shape: (532233, 2)


,text,metadata
0,# விக்கிப்பீடியா:கலந்துரையாடல்\n\n***இங்கு தமி...,{'Header 1': 'விக்கிப்பீடியா:கலந்துரையாடல்'}
1,# விக்கிப்பீடியா:கலந்துரையாடல்\n\n## முந்தைய க...,"{'Header 1': 'விக்கிப்பீடியா:கலந்துரையாடல்', '..."
2,# விக்கிப்பீடியா:கலந்துரையாடல்\n\n## பயனர் கரு...,"{'Header 1': 'விக்கிப்பீடியா:கலந்துரையாடல்', '..."
3,# விக்கிப்பீடியா:கலந்துரையாடல்\n\n## பகுப்புத்...,"{'Header 1': 'விக்கிப்பீடியா:கலந்துரையாடல்', '..."
4,# விக்கிப்பீடியா:கலந்துரையாடல்\n\n## மலேசியாவி...,"{'Header 1': 'விக்கிப்பீடியா:கலந்துரையாடல்', '..."


## Analyze chunks

In [8]:
# Calculate statistics
chunks_df['text_length'] = chunks_df['text'].str.len()
chunks_df['word_count'] = chunks_df['text'].str.split().str.len()

print("Chunk statistics:")
print(f"Total chunks: {len(chunks_df)}")
print(f"\nText length (characters):")
print(chunks_df['text_length'].describe())
print(f"\nWord count:")
print(chunks_df['word_count'].describe())

# Check how many chunks have header metadata
chunks_with_headers = chunks_df[chunks_df['metadata'].apply(lambda x: len(x) > 0)]
print(f"\nChunks with header metadata: {len(chunks_with_headers)} ({len(chunks_with_headers)/len(chunks_df)*100:.1f}%)")

Chunk statistics:
Total chunks: 532233

Text length (characters):
count    532233.000000
mean        692.793070
std         956.501444
min           5.000000
25%         241.000000
50%         437.000000
75%         758.000000
max        7999.000000
Name: text_length, dtype: float64

Word count:
count    532233.000000
mean         90.099361
std         187.534264
min           2.000000
25%          29.000000
50%          51.000000
75%          89.000000
max        3244.000000
Name: word_count, dtype: float64

Chunks with header metadata: 532233 (100.0%)


In [9]:
# Display sample chunks with their metadata
print("Sample chunks with metadata:\n")
for i in range(min(5, len(chunks_df))):
    print(f"\n{'='*80}")
    print(f"Chunk {i+1}")
    print(f"{'='*80}")
    print(f"Length: {chunks_df.iloc[i]['text_length']} chars, {chunks_df.iloc[i]['word_count']} words")
    print(f"Metadata: {chunks_df.iloc[i]['metadata']}")
    print(f"\nText:\n{chunks_df.iloc[i]['text'][:300]}...")


Sample chunks with metadata:


Chunk 1
Length: 571 chars, 53 words
Metadata: {'Header 1': 'விக்கிப்பீடியா:கலந்துரையாடல்'}

Text:
# விக்கிப்பீடியா:கலந்துரையாடல்

***இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் பொதுவான கருத்துக்கள், பாராட்டுக்கள் மற்றும் ஆலோசனைகளைத் தெரிவிக்கலாம். உங்கள் கருத்துகளுக்கு மற்ற விக்கிப்பீடியர்கள் பதில் அளிப்பார்கள். தகுந்த ஆலோசனைகளை உடனே செயற்படுத்தவும் செய்வோம். விக்கிப்பீடியா திட்டத்தின் வளர்ச்சி...

Chunk 2
Length: 206 chars, 22 words
Metadata: {'Header 1': 'விக்கிப்பீடியா:கலந்துரையாடல்', 'Header 2': 'முந்தைய கலந்துரையாடல்கள்'}

Text:
# விக்கிப்பீடியா:கலந்துரையாடல்

## முந்தைய கலந்துரையாடல்கள்

- **தொகுப்பு 01** (மயூரநாதனின் முதற் குறிப்பு!, தமிழ் விக்கிப்பீடியா 1000 கட்டுரைகள்.)  
- **தொகுப்பு 02**
மிகவும் அருமையான முயற்சி வாழ்த்துக்கள்...

Chunk 3
Length: 2372 chars, 257 words
Metadata: {'Header 1': 'விக்கிப்பீடியா:கலந்துரையாடல்', 'Header 2': 'பயனர் கருத்துகள்'}

Text:
# விக்கிப்பீடியா:கலந்துரையாடல்

## பயனர் கருத்துகள்

உங்கள் கருத்துகளை இதன் கீழ் இடவு

## Save chunked dataset

In [10]:
# Drop temporary columns before saving
chunks_df_final = chunks_df.drop(columns=['text_length', 'word_count'])

# Save as parquet
chunks_df_final.to_parquet(OUTPUT_FILE, index=False)
print(f"Saved {len(chunks_df_final)} chunks to {OUTPUT_FILE}")

# Also save as JSON for inspection
json_output = OUTPUT_FILE.replace('.parquet', '.jsonl')
chunks_df_final.to_json(json_output, orient='records', lines=True, force_ascii=False)
print(f"Also saved to {json_output}")

Saved 532233 chunks to data/tawiki_chunked.parquet
Also saved to data/tawiki_chunked.jsonl
